In [25]:
import numpy as np
X = np.array([
    [3, 2, 1],
    [2, 4, 5],
    [1, 2, 3],
    [0, 2, 5]
])
mean_X = np.mean(X, axis=0)
print(mean_X)
print()
# 1.b
X_centered = X - mean_X
Q = np.cov(X_centered, rowvar=False, bias=True)
print("Sample covariance matrix Q:")
print(Q)

[1.5 2.5 3.5]

Sample covariance matrix Q:
[[ 1.25  0.25 -1.25]
 [ 0.25  0.75  0.75]
 [-1.25  0.75  2.75]]


In [28]:
# 1.c
eig_vals, eig_vecs = np.linalg.eigh(Q)
print("Eigenvalues:", eig_vals)
print("Eigenvectors:\n", eig_vecs)

Eigenvalues: [0.01495506 1.1733803  3.56166464]
Eigenvectors:
 [[ 0.59363515  0.66677184 -0.45056922]
 [-0.66472154  0.72187235  0.19247228]
 [ 0.45358856  0.18524476  0.87174641]]


In [29]:
# 1.d
Z = X_centered @ eig_vecs
print("PCA coefficients:\n", Z)

PCA coefficients:
 [[ 0.0888421   0.17610969 -2.95145599]
 [-0.0198819   1.69406159  1.37104342]
 [-0.19125108 -0.78694448 -0.30682473]
 [ 0.12229089 -1.0832268   1.8872373 ]]


In [16]:
# 1.e
X_reconstructed = Z @ eig_vecs.T + mean_X
print("Reconstructed samples:\n", X_reconstructed)

Reconstructed samples:
 [[3.00000000e+00 2.00000000e+00 1.00000000e+00]
 [2.00000000e+00 4.00000000e+00 5.00000000e+00]
 [1.00000000e+00 2.00000000e+00 3.00000000e+00]
 [2.22044605e-16 2.00000000e+00 5.00000000e+00]]


In [30]:
# 1.f
idx = np.argsort(eig_vals)[-2:]
Z_approx = Z[:, idx]
eigvecs_approx = eig_vecs[:, idx]
X_approx = Z_approx @ eigvecs_approx.T + mean_X
print("Approximation using two largest eigenvalues:\n", X_approx)

Approximation using two largest eigenvalues:
 [[ 2.94726021  2.05905526  0.95970224]
 [ 2.0118026   3.98678407  5.0090182 ]
 [ 1.11353336  1.87287129  3.0867493 ]
 [-0.07259617  2.08128939  4.94453025]]


In [31]:
# 1.g
reconstruction_error = np.sum((X_centered - Z_approx @ eigvecs_approx.T) ** 2)
skipped_coefficients_sum = np.sum(Z[:, :-2] ** 2)
print("Reconstruction error sum of squares:", reconstruction_error)
print("Sum of squares of skipped PCA coefficients:", skipped_coefficients_sum)

Reconstruction error sum of squares: 0.059820245731225935
Sum of squares of skipped PCA coefficients: 0.059820245731225956


In [33]:
# 2.a
mu = np.array([1, 0, 2])
x = np.array([2, 3, 4])
v1 = np.array([1, 1, 0]) / np.sqrt(2)
v2 = np.array([1, -1, 0]) / np.sqrt(2)

xc = x - mu
c1 = np.dot(xc, v1)
c2 = np.dot(xc, v2)
print("PC coefficients:", c1, c2)

PC coefficients: 2.82842712474619 -1.414213562373095


In [37]:
# 2.b
x_hat = mu + c1 * v1 + c2 * v2
print("Approximation of x:", x_hat)

Approximation of x: [2. 3. 2.]


In [39]:
# 2.c
error = np.sum((x - x_hat) ** 2)
print("Approximation error:", error)

Approximation error: 4.0


In [40]:
# 3 1~n components accuracy test
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fitting_data_with_pcs(X, y):
    Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=0.25, shuffle=False)

    best_accuracy = 0
    best_n_components = 0

    for i in range(1, Xtr.shape[1] +1):
        pca = PCA(n_components=i)
        Ztr = pca.fit_transform(Xtr)
        Zts = pca.transform(Xts)

        clf = LogisticRegression()
        clf.fit(Ztr, ytr)

        y_hat = clf.predict(Zts)
        acc = accuracy_score(yts, y_hat)
        if acc > best_accuracy:
            best_accuracy = acc
            best_n_components = i

    print("Best n of PC:", best_n_components)
    print("Best accuracy:", best_accuracy)

In [41]:
X = np.array([
    [2.5, 3.0, 1.5, 2.8],
    [1.5, 2.3, 3.0, 1.2],
    [3.1, 3.5, 4.0, 4.2],
    [4.5, 4.1, 2.0, 3.3],
    [1.0, 1.5, 2.5, 2.0],
    [2.2, 2.7, 3.1, 1.5],
    [3.5, 3.3, 1.8, 2.7],
    [2.1, 2.8, 4.0, 3.5],
    [4.0, 3.8, 2.5, 3.9],
    [1.8, 1.9, 3.4, 2.6],
    [2.7, 3.5, 1.9, 2.4],
    [1.4, 2.1, 2.9, 1.8],
])

y = np.array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1])
fitting_data_with_pcs(X, y)

Best n of PC: 2
Best accuracy: 0.6666666666666666


In [42]:
# 4

def pc_with_images(X):
    X_reshaped = X.reshape(1000, -1)
    Xtr = X_reshaped[:500]
    Xts = X_reshaped[500:]

    pca = PCA(n_components=5)
    pca.fit_transform(Xtr)
    pca.transform(Xts)

    Zts = pca.transform(Xts)
    Xts_approx = pca.inverse_transform(Zts)

    Xts_approx_images = Xts_approx.reshape(500, 28, 28)
    return Xts_approx_images


In [43]:
# 5
from scipy.linalg import svd

def pcs_using_svds(X):
    mean_X = np.mean(X, axis=0)
    print("Mean:", mean_X)

    X_centered = X - mean_X
    U, s, Vtr = svd(X_centered, full_matrices=False)
    var_explained = np.cumsum(s**2) / np.sum(s**2)
    num_pcs = np.argmax(var_explained >= 0.9) + 1

    print(f"Number of PCs for 90% variance: {num_pcs}")

    U_reduced = U[:, :num_pcs]
    s_reduced = s[:num_pcs]
    Vtr_reduced = Vtr[:num_pcs, :]

    X_approx = (U_reduced * s_reduced) @ Vtr_reduced + mean_X
    return X_approx

